In [10]:
import spacy
import os
import spacy
import pandas as pd
import spacy
import nltk
import string
from spacy import displacy
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [11]:
from spacy.lang.en import English
import gensim.downloader as api

In [12]:
glove_model = api.load("glove-wiki-gigaword-50")
import spacy

In [13]:
import pytextrank
import neuralcoref

In [14]:
# ---------------------------------------
# python -m spacy download en_core_web_sm
# python -m spacy download en_core_web_md
# ---------------------------------------
import spacy

In [15]:
def get_data_raw():
    with open("data.txt",'r') as fh:
        txt = fh.readlines()
    return str(txt[0])


In [16]:
text = get_data_raw()

## 1. Is this a crash event?
## 2. How many vehicles were involved in the crash?
## 3. What were the types of the vehicles?
## 4. What was the type of the collision?
## 5. Where did this happen? (highway/city, straight road/intersection)

In [17]:
nlp =  spacy.load("en_core_web_md")
try:
    nlp.add_pipe(nlp.create_pipe('sentencizer')) 
except:
    pass
try:
    neuralcoref.add_to_pipe(nlp)
except:
    pass

doc = nlp(text)

In [18]:

from spacy.matcher import Matcher
nounPhrase_posExclude = ['PRON', 'DET', 'PART', 'ADJ']

with doc.retokenize() as retokenizer:
  
    for noun_phrase in list(doc.noun_chunks):
        cur_span = []
       
        for item in noun_phrase:
            if item.pos_ not in nounPhrase_posExclude:
                cur_span.append(item)
        if len(cur_span) > 1:       
            _span = doc[cur_span[0].i : cur_span[-1].i ]
        
            retokenizer.merge(
               _span
            )
    
    # ================================
    # Handle terms like "t-boned", "rear-ended", "side-swiped"
    # ================================
    pattern = [{"POS": "NOUN"}, {"ORTH":'-'} , {"POS": "VERB"}]
    matcher = Matcher(nlp.vocab)
    matcher.add( 'NV' ,None,pattern)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id] 
        span = doc[start:end]  
#         print(match_id, string_id, start, end, span.text)

        retokenizer.merge(span)

In [19]:
sentences = [sent.string.strip() for sent in doc.sents]

In [20]:
# Tokens
tokens = [token for token in doc]

In [108]:
for sentence in doc.sents:
    print('----')
    for token in sentence:
        print(token.text,' Is stop word/punctuation ?',token.is_stop or token.is_punct , ' POS ', token.pos_, 'Lemma ', token.lemma_)

----
I  Is stop word/punctuation ? True  POS  PRON Lemma  -PRON-
pulled  Is stop word/punctuation ? False  POS  VERB Lemma  pull
out  Is stop word/punctuation ? True  POS  SCONJ Lemma  out
of  Is stop word/punctuation ? True  POS  ADP Lemma  of
my  Is stop word/punctuation ? True  POS  DET Lemma  -PRON-
parking  Is stop word/punctuation ? False  POS  NOUN Lemma  park
spot  Is stop word/punctuation ? False  POS  NOUN Lemma  spot
,  Is stop word/punctuation ? True  POS  PUNCT Lemma  ,
turned  Is stop word/punctuation ? False  POS  VERB Lemma  turn
right  Is stop word/punctuation ? False  POS  ADV Lemma  right
out  Is stop word/punctuation ? True  POS  SCONJ Lemma  out
of  Is stop word/punctuation ? True  POS  ADP Lemma  of
the  Is stop word/punctuation ? True  POS  DET Lemma  the
parking  Is stop word/punctuation ? False  POS  NOUN Lemma  park
lot  Is stop word/punctuation ? False  POS  NOUN Lemma  lot
,  Is stop word/punctuation ? True  POS  PUNCT Lemma  ,
then  Is stop word/punctuation

In [28]:
i= 0 
from pathlib import Path

for sentence in doc.sents:
    i +=1
    print(sentence)
    # displaying tokens with their POS tags
    
#     displacy.render(sentence,style='dep',jupyter=True)
    svg = displacy.render(sentence, style="dep", jupyter=False)

    output_path = Path("sentence{}.svg".format(i))
    print(output_path)
    output_path.open("w", encoding="utf-8").write(svg)

I pulled out of my parking spot, turned right out of the parking lot, then proceeded to the next stop sign.
sentence1.svg
While looking both ways I noticed a white truck in the far distance and proceeded to roll forward and take the left turn.
sentence2.svg
As I turned I was t-boned on the front driver side by the white truck, which turned out to be a Ford F-450 commercial truck.
sentence3.svg
All I remember hearing was the rumbling of crushing metal.
sentence4.svg
Needless to say my beloved car that I had worked so hard for appeared to be totaled.
sentence5.svg
All of my airbags deployed, my front windshield was smashed  in, the driver’s window was broken, my key jammed in the ignition, and I was unconscious.
sentence6.svg
When I came to my first reaction was hysterics.
sentence7.svg
I looked around for a minute, heard voices talking to me, then I felt an unbearable pain in my right hand.
sentence8.svg
As I looked down at my hand I noticed that it appeared to be separated from my wris

In [27]:
for sentence in doc.sents:
    displacy.render(sentence, style="ent")

/home/ddatta/anaconda3/envs/text/lib/python3.7/site-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [111]:
nounPhrase_posExclude = ['PRON', 'DET', 'PART']
candidate_spans = []
for sentence in doc.sents:
    for spanChunk in sentence.noun_chunks:
        print(spanChunk.text)
        # process the span
        if len(spanChunk.ents)>0 :
            print('entity', spanChunk.ents, spanChunk.ents[0].label_ )
        cur_span = []
        
        for item in spanChunk:
            if item.pos_ not in nounPhrase_posExclude:
                print('>>', item.lemma_)
                cur_span.append(item)
        if len(cur_span) > 0:
            candidate_spans.append(cur_span)
        
        print('---')
        
        

I
---
my parking spot
>> park
>> spot
---
the parking lot
>> park
>> lot
---
the next stop sign
>> next
>> stop
>> sign
---
both ways
>> way
---
I
---
a white truck
>> white
>> truck
---
the far distance
>> far
>> distance
---
the left turn
>> left
>> turn
---
I
---
I
---
the front driver side
>> front
>> driver
>> side
---
the white truck
entity [the white truck] ORG
>> white
>> truck
---
a Ford F-450 commercial truck
entity [Ford F-450 commercial] PRODUCT
>> Ford F-450 commercial
>> truck
---
I
---
the rumbling
>> rumbling
---
metal
>> metal
---
my beloved car
>> beloved
>> car
---
I
---
my airbags
>> airbag
---
my front windshield
>> front
>> windshield
---
the driver’s window
>> driver’s
>> window
---
the ignition
>> ignition
---
I
---
I
---
my first reaction
entity [first] ORDINAL
>> first
>> reaction
---
I
---
a minute
>> minute
---
voices
>> voice
---
me
---
I
---
an unbearable pain
>> unbearable
>> pain
---
my right hand
>> right
>> hand
---
I
---
my hand
>> hand
---
I
---
it
-

In [112]:
# add PyTextRank to the spaCy pipeline
tr = pytextrank.TextRank()
try:
    nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)
except:
    pass

doc1 = nlp(text)

# examine the top-ranked phrases in the document
for p in doc1._.phrases:
    print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
    print(p.chunks)

0.0711     1  first
[first]
0.0687     1  a ford f-450 commercial truck
[a Ford F-450 commercial truck]
0.0679     1  the front driver side
[the front driver side]
0.0630     1  voices
[voices]
0.0577     1  ford
[Ford]
0.0577     1  f-450
[F-450]
0.0556     1  my front windshield
[my front windshield]
0.0541     1  metal
[metal]
0.0532     1  the next stop sign
[the next stop sign]
0.0516     1  a white truck
[a white truck]
0.0498     1  my first reaction
[my first reaction]
0.0496     1  my beloved car
[my beloved car]
0.0468     1  an unbearable pain
[an unbearable pain]
0.0452     1  two other cars
[two other cars]
0.0441     1  my parking spot
[my parking spot]
0.0438     1  the parking lot
[the parking lot]
0.0423     1  the far distance
[the far distance]
0.0418     1  the left turn
[the left turn]
0.0415     1  my right hand
[my right hand]
0.0408     2  the white truck
[the white truck, the white truck]
0.0340     1  the rumbling
[the rumbling]
0.0339     1  the driver’s wind

In [46]:
glove_model

In [47]:
glove_model.most_similar('car')
glove_model.most_similar_cosmul('yield')

[('yields', 0.96498042345047),
 ('yielding', 0.9228639006614685),
 ('yielded', 0.8758969902992249),
 ('6.25', 0.8733323216438293),
 ('3.75', 0.8696194887161255),
 ('maturity', 0.8650699257850647),
 ('benchmark', 0.8637473583221436),
 ('bond', 0.8622376918792725),
 ('higher', 0.8615794777870178),
 ('5.75', 0.8570329546928406)]

In [113]:
print(doc._.coref_clusters)

[my right hand: [my right hand, my hand], As: [As, it]]


In [21]:
# adposition : ADP : prpeopsition + postposition ; occurs with anoun phrase

In [114]:
from spacy.symbols import nsubj, amod, VERB, ADJ, ADP, DET
verbs = set()
for sentence in doc.sents:
    print(sentence)
    for cand_subject in sentence:
        if cand_subject.dep == nsubj and cand_subject.head.pos == VERB:
            verbs.add(cand_subject.head)
            print('>', cand_subject.head)
#         if cand_subject.dep == amod  or ( cand_subject.pos == ADJ or cand_subject.head.pos == ADP):
#             print('>>', cand_subject.lemma_)
print(verbs)

I pulled out of my parking spot, turned right out of the parking lot, then proceeded to the next stop sign.
> pulled
While looking both ways I noticed a white truck in the far distance and proceeded to roll forward and take the left turn.
> noticed
As I turned I was t-boned on the front driver side by the white truck, which turned out to be a Ford F-450 commercial truck.
> turned
> turned
All I remember hearing was the rumbling of crushing metal.
> remember
Needless to say my beloved car that I had worked so hard for appeared to be totaled.
> worked
All of my airbags deployed, my front windshield was smashed  in, the driver’s window was broken, my key jammed in the ignition, and I was unconscious.
> deployed
> jammed
When I came to my first reaction was hysterics.
> came
I looked around for a minute, heard voices talking to me, then I felt an unbearable pain in my right hand.
> looked
> talking
> felt
As I looked down at my hand I noticed that it appeared to be separated from my wrist.

In [115]:
for sentence in doc.sents:
    print('----')
    print(sentence)
    print('---->')
    for word in sentence:
        if word.dep_ in ("xcomp", "ccomp"):
            subtree_span = doc[ word.left_edge.i : word.right_edge.i + 1]
            print(subtree_span.text, "|", subtree_span.root.text, subtree_span.root.pos_)

----
I pulled out of my parking spot, turned right out of the parking lot, then proceeded to the next stop sign.
---->
----
While looking both ways I noticed a white truck in the far distance and proceeded to roll forward and take the left turn.
---->
to roll forward and take the left turn | roll VERB
----
As I turned I was t-boned on the front driver side by the white truck, which turned out to be a Ford F-450 commercial truck.
---->
to be a Ford F-450 commercial truck | be AUX
----
All I remember hearing was the rumbling of crushing metal.
---->
hearing | hearing VERB
----
Needless to say my beloved car that I had worked so hard for appeared to be totaled.
---->
to say my beloved car that I had worked so hard for appeared to be totaled | say VERB
to be totaled | totaled VERB
----
All of my airbags deployed, my front windshield was smashed  in, the driver’s window was broken, my key jammed in the ignition, and I was unconscious.
---->
All of my airbags deployed | deployed VERB
----
Wh

In [116]:
# -------------------------------
# Count the number of vehicles
# -------------------------------
# Assume a narrative text
# -------------------------------

from nltk.stem.porter import *
stemmer = PorterStemmer()

vehicle_seed_words = ['car','vehicle','automoile','bus','minivan','truck','motorcycle','minivan', 'bike','cycle']
vehicle_synonyms = [ ]
for veh_w in vehicle_seed_words: 
    try:
        vehicle_synonyms.extend([ lemmatizer.lemmatize(_[0])  for _ in  glove_model.most_similar(veh_w)])
    except:
        pass
    
from collections import Counter
vehicle_synonyms= [ v for v,count in Counter(vehicle_synonyms).items() if count > 1 ]
vehicle_synonyms
# Human exper based pruning
vehicle_synonyms.remove('driver')
vehicle_synonyms.remove('driving')
vehicle_synonyms.remove('parked')
vehicle_synonyms.remove('dealership')
def indicates_vehicle(cur_word):
    global vehicle_synonyms
    if cur_word in vehicle_synonyms:
        return True
    return False

In [117]:
# ----------------------------------
# A simple rule based detector
# ----------------------------------
# Disadvantages ? 
# 1. Negation handling is difficult
# ----------------------------------

In [118]:
seen_phrases = []

In [119]:
for sentence in doc.sents:
    print( '>', sentence )
    for word in sentence:
        # ----------------
        # Rule to check for the narrator's reference to vehicles
        # e.g. "our truck" 
        # ----------------
        if word.dep_ == "poss" and word.pos_ == "DET": 
            # words like my, ours, etc
            if indicates_vehicle(word.head.lemma_):
                print('[det]',  word.head.pos_, word.head)
                cur_phrase = ('[det]',word.head.lemma_)
                if cur_phrase not in seen_phrases:
                    seen_phrases.append(cur_phrase)
        # Handle phrases like : "the * car"
        elif word.dep_ == "det" and word.pos_ == "DET":
             if indicates_vehicle(word.head.lemma_) :
                print('[det]', word.head,  word.head.pos_)
                cur_phrase = ('[det]',  word.head.lemma_)
                seen_phrases.append(cur_phrase)
        # Handle phrases like "three more cars .."
        elif word.pos == 'NUM' or word.dep_=='nummod':
            # check if there is no poss child of the head like "ours"
            flag = True
            indicates_vehicle(word.head.lemma_)
            for c in word.head.children:
                if c!=word and c.dep_== "poss":
                    flag = False
            if flag:
                cur_phrase = ('[COUNT:{}]'.format(word.text), word.head.lemma_ )
                seen_phrases.append(cur_phrase)
            

> I pulled out of my parking spot, turned right out of the parking lot, then proceeded to the next stop sign.
> While looking both ways I noticed a white truck in the far distance and proceeded to roll forward and take the left turn.
[det] truck NOUN
> As I turned I was t-boned on the front driver side by the white truck, which turned out to be a Ford F-450 commercial truck.
[det] truck PROPN
[det] truck NOUN
> All I remember hearing was the rumbling of crushing metal.
> Needless to say my beloved car that I had worked so hard for appeared to be totaled.
[det] NOUN car
> All of my airbags deployed, my front windshield was smashed  in, the driver’s window was broken, my key jammed in the ignition, and I was unconscious.
> When I came to my first reaction was hysterics.
> I looked around for a minute, heard voices talking to me, then I felt an unbearable pain in my right hand.
> As I looked down at my hand I noticed that it appeared to be separated from my wrist.
> There were two other c

In [120]:
if len(set(seen_phrases)) > 2:
    print("Two or more")
elif len(set(seen_phrases)) == 1:
    print('single')

Two or more


In [121]:
# -----------------------------------
# Create a graph 
# -----------------------------------
import networkx as nx


In [122]:
# ====================================
# Keep refereal types as determinants
# types  
# 1 {'my','ours'}
# 2 {their}
# 3 {the, that a, an}
# ====================================

In [123]:
def get_ref_type(_txt_):
    
    if _txt_ in ['my','our','mine']:
        return 1
    elif _txt_ in ['their']:
        return 2
    elif _txt_ in ['the', 'that', 'a', 'an']:
        return 3
    else:
        return 0
    
def indicates_contact(text):
    candidates = ['rear-ended','hit','strike','crash','struck','collide','t-boned']
    return text in candidates



In [124]:
import spacy
from nltk.corpus import wordnet

In [125]:
glove_model.most_similar('rear-end')

[('chain-driven', 0.7046515345573425),
 ('engine-powered', 0.6801375150680542),
 ('atr-42', 0.6755519509315491),
 ('mid-air', 0.6702165603637695),
 ('motor-driven', 0.6601287722587585),
 ('out-of-control', 0.6502394676208496),
 ('parkinsonian', 0.6491718888282776),
 ('vbied', 0.6469529867172241),
 ('basepath', 0.6467806100845337),
 ('carry-over', 0.6454182863235474)]

In [126]:
class node_obj:
    _id = -1
    
    def __init__(self, ref, txt):
        
        self.id = node_obj._id + 1
        node_obj._id += 1
        
        self.text = txt
        self.ref = ref
        self.object_type = 'entity'
        self.attr = []
        return 
    
        
    def add_attr(self, attr):
        self.attr.append(attr)
        

In [127]:
def add_node(_ref_, _txt_):
    _exists = False
    global node_list
    for obj in node_list :
        if obj.ref == _ref_ and obj.text == _txt_:  
            _exists = True
            return obj
    
    if not _exists:
        obj =  node_obj(_ref_ , _txt_)
        node_list.append(
           obj
        )
    return obj
    

In [156]:
node_list = []
nounPhrase_posExclude = ['PRON', 'DET', 'PART']
candidate_spans = []
disp_sentence = []

# ===============================
# Process each sentence
# ===============================
for sentence in doc.sents:         
    disp_sentence = []
    disp_sentence.append({'text':sentence.text,'ents' :[] })
    objs = []
    for spanChunk in sentence.noun_chunks:
        for word in spanChunk:
            if word.dep_ == "poss" and word.pos_ == "DET": 
                head_token = word.head
                # words like my, ours, etc
                if head_token.pos_ =='NOUN' and indicates_vehicle(head_token.lemma_):
                    _ref_ = get_ref_type( word.lemma_)
                    _txt_ =  head_token.lemma_
                    obj = add_node( _ref_, head_token.lemma_)
                    obj.type = 'vehicle'
                    objs.append(obj)
                    continue      
                              
            elif word.pos_ == "DET" : 
                head_token = word.head
                
                if (head_token.pos_ == 'PROPN' or head_token.pos_ == 'NOUN') and indicates_vehicle(head_token.lemma_):
                    
                    _ref_ = get_ref_type(word.lemma_)
                    _txt_ =  head_token.lemma_
                    obj = add_node( _ref_, head_token.lemma_)
                    obj.type = 'vehicle'
                    objs.append(obj)
                    continue   
                
        # ------------------------
        # Identify the entities 
        # ------------------------
        if len(spanChunk.ents) > 0 :
            for c_ent in spanChunk.ents:
                # identify products :e.g (Chevrolet) Impala
                if c_ent.label_ == 'PRODUCT' and (doc[c_ent.start].head.pos_=='NOUN' or doc[c_ent.start].head.pos_=='PROPN'): 
                    head_token = doc[c_ent.start].head
                    _ref_ = get_ref_type(head_token.left_edge.text) # the determinant
                    _txt_ = head_token.lemma_
                    # check if the entity has been added
                    obj =  add_node(_ref_, _txt_)
                    obj.add_attr((c_ent.label_ ,c_ent.text))
                    objs.append(obj)
                    
                # terms such as "first bus" or "second car" can be assumed that the entity has been previously referenced.
                elif c_ent.label_ == 'ORDINAL': 
                    head_token = doc[c_ent.start].head
                    if head_token.pos_ == 'NOUN':
                         pass
                    
                # terms such as "two" or "three"    
                elif c_ent.label_ == 'CARDINAL':
                    head_token = doc[c_ent.start].head
                    pass
                
                # terms such as "4:30 p.m."
                elif c_ent.label_ == 'TIME':
                    token = doc[c_ent.start]
                    pass
           
        # ---- Visualize 
        cur_span = []
        for item in spanChunk:
            if item.pos_ not in nounPhrase_posExclude:
                cur_span.append(item)

        if len(cur_span) > 0:
            candidate_spans.append(cur_span)
            disp_sentence[0]['ents'].append({
                'start': cur_span[0].idx - sentence[0].idx ,
                'end': cur_span[-1].idx - sentence[0].idx + len( cur_span[-1].text),
                'label': ''
                }
            )
    for word in sentence:
        if word.pos_=='VERB' and indicates_contact(word.lemma_):
            print('VERB indicating collision event :: ', word.lemma_)
        
        
    colors = {"": "linear-gradient(10deg, #00FF11, #FFF252)"}
    options = {"ents": [""], "colors": colors}
    displacy.render(disp_sentence, style='ent', manual=True, options=options)           
           





VERB indicating collision event ::  t-boned


In [130]:
for _ in node_list: 
    print(_.text,_.type,_.attr)

truck vehicle [('PRODUCT', 'Ford F-450 commercial')]
car vehicle []


In [143]:

for sentence in doc.sents: 
    flag = False
    disp_sentence = [{'text':sentence.text,'ents' :[] }]
    for word in sentence:
        if word.pos_=='VERB' and indicates_contact(word.lemma_):
            print('VERB indicating collision event :: ', word.lemma_)
            flag = True
            disp_sentence[0]['ents'].append({
                'start': word.idx - sentence[0].idx ,
                'end':   word.idx - sentence[0].idx + len( word.text),
                'label': ''
                }
            )
    if flag:
        colors = {"": "linear-gradient(10deg, #00FF00, #00FFFF)"}
        options = {"ents": [""], "colors": colors}        
        displacy.render(disp_sentence, style='ent', manual=True, options=options)         

VERB indicating collision event ::  t-boned
